In [48]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [52]:
# haal namen van ggd_regio's op
gdf = gpd.read_file('ggd_regios.shp')
gdf.statnaam

0                                         GGD Groningen
1                                           GGD Drenthe
2                                        GGD IJsselland
3                                      GGD Regio Twente
4                         GGD Noord- en Oost-Gelderland
5     Veiligheids- en Gezondheidsregio Gelderland-Mi...
6                                   GGD Gelderland-Zuid
7                                         GGD Flevoland
8                                     GGD Regio Utrecht
9                                  GGD Hollands-Noorden
10                                     GGD Kennemerland
11                                        GGD Amsterdam
12                              GGD Gooi en Vechtstreek
13                                  GGD Hollands-Midden
14                               GGD Rotterdam-Rijnmond
15                        Dienst Gezondheid & Jeugd ZHZ
16                                          GGD Zeeland
17                                     GGD West-

In [58]:
np.random.seed(42)

models = [1, 6]
ggd_regios = gdf.statnaam.values
uitkomstmaten = ['Gezondheid Algemeen',
               'Totale Zorgkosten']

df = pd.DataFrame(columns=['uitkomstmaat', 'model', 'ggd_regio', 'b', 'll', 'ul'])

for maat in uitkomstmaten:
    for model in models:

        # voor gezondheidsmaten zijn beta's als probabilities
        beta = np.random.normal(.4, .1, size=25)
        # bij model 3 en later zijn de probabilities van regio's minder verschillend
        if 'gezondheid' in maat.lower() and model >= 3:
            beta = np.random.normal(.24, .01, size=25)
            
        # voor kosten uitkomstmaten zijn beta's als marginale kosten in euro's
        if 'kosten' in maat.lower():
            beta = np.random.normal(2200, 400, size=25)
            if model >= 3:
                beta = np.random.normal(2200, 50, size=25)
        
        se = beta * np.random.normal(0.05, 0.01)
        
        lower_limit = beta - 1.96 * se
        upper_limit = beta + 1.96 * se
        
        model_df = pd.DataFrame({'uitkomstmaat': [maat] * 25,
                                'model': [model] * 25,
                                'ggd_regio': ggd_regios,
                                'b': beta,
                                'll': lower_limit,
                                'ul': upper_limit})
        
        df = pd.concat([df, model_df], axis=0)

df

,uitkomstmaat,model,ggd_regio,b,ll,ul
0,Gezondheid Algemeen,1,GGD Groningen,0.449671,0.404626,0.494717
1,Gezondheid Algemeen,1,GGD Drenthe,0.386174,0.347489,0.424858
2,Gezondheid Algemeen,1,GGD IJsselland,0.464769,0.418211,0.511327
3,Gezondheid Algemeen,1,GGD Regio Twente,0.552303,0.496977,0.607629
4,Gezondheid Algemeen,1,GGD Noord- en Oost-Gelderland,0.376585,0.338861,0.414309
...,...,...,...,...,...,...
20,Totale Zorgkosten,6,GGD Limburg-Noord,2202.910436,1941.525155,2464.295717
21,Totale Zorgkosten,6,GGD Zuid-Limburg,2142.851485,1888.592470,2397.110500
22,Totale Zorgkosten,6,GGD Haaglanden,2217.889368,1954.726769,2481.051967
23,Totale Zorgkosten,6,GGD Fryslân,2228.039226,1963.672301,2492.406152


In [59]:
df.dtypes

uitkomstmaat     object
model            object
ggd_regio        object
b               float64
ll              float64
ul              float64
dtype: object

In [60]:
# dat is het begin, maar dan moet er nog per regio bekeken worden welke andere regio's significant verschillen door de lower en upper limits te vergelijken

results = pd.DataFrame(columns=['uitkomstmaat', 'model', 'ggd_regio', 'b', 'll', 'ul', 'referentie_regio', 'beta_referentie_regio', 'goedkoper_dan_ref_regio', 'duurder_dan_ref_regio', 'verschil'])


# uitkomstmaat = 'Totale Zorgkosten'
# model = 1
# ggd_regio = 'GGD regio 14'



for uitkomstmaat in df.uitkomstmaat.unique():
    for model in df.model.unique():
        for ggd_regio in df.ggd_regio.unique():
            ggd_df = df.loc[(df.uitkomstmaat == uitkomstmaat) &
                           (df.model == model), ].copy()

            ggd_df['referentie_regio'] = ggd_regio
            ggd_df['beta_referentie_regio'] = ggd_df.loc[ggd_df.ggd_regio == ggd_regio, 'b'].values[0]

            ll_ref_regio = ggd_df.loc[ggd_df.ggd_regio == ggd_regio, 'll'].values[0]
            ul_ref_regio = ggd_df.loc[ggd_df.ggd_regio == ggd_regio, 'ul'].values[0]

            ggd_df['goedkoper_dan_ref_regio'] = ggd_df['ul'] < ll_ref_regio
            ggd_df['duurder_dan_ref_regio'] = ggd_df['ll'] > ul_ref_regio


            ggd_df['verschil'] = ggd_df['b'] - ggd_df['beta_referentie_regio']
            # niet-significante verschillen moeten grijs worden op kaart, daar maken we het verschil nul
            ggd_df.loc[(~ggd_df.goedkoper_dan_ref_regio) & (~ggd_df.duurder_dan_ref_regio), 'verschil'] = 0            

            results = pd.concat([results, ggd_df])

results.to_csv('simulated_ggd_regio_data.csv', index=False)
results


# per maat
# per model
# per regio:
    # maak een kopie van df met deze maat en model:
    # voeg de kolommen 'ref-regio', 'p-waarde' en 'verschil' toe. En evt lower_limit_ref_regio en upper_limit_ref_regio
        # wanneer de upper_limit van andere regio lager ligt dan lower limit van de ref-regio:
            # dan is die andere regio significant goedkoper
            # noteer significantie ja/nee (kolom 'p-waarde')
            # noteer verschil in % of EUR (kolom 'verschil')
        # wanneer de lower_limit van andere regio hoger ligt dan de upper_limit van de ref-regio:
            # dan is die andere regio significant duurder
            # noteer significantie ja/nee (kolom 'p-waarde')
            # noteer verschil in % of EUR (kolom 'verschil')
    # voeg de kopie toe aan nieuwe (hele lange) tabel





,uitkomstmaat,model,ggd_regio,b,ll,ul,referentie_regio,beta_referentie_regio,goedkoper_dan_ref_regio,duurder_dan_ref_regio,verschil
0,Gezondheid Algemeen,1,GGD Groningen,0.449671,0.404626,0.494717,GGD Groningen,0.449671,False,False,0.000000
1,Gezondheid Algemeen,1,GGD Drenthe,0.386174,0.347489,0.424858,GGD Groningen,0.449671,False,False,0.000000
2,Gezondheid Algemeen,1,GGD IJsselland,0.464769,0.418211,0.511327,GGD Groningen,0.449671,False,False,0.000000
3,Gezondheid Algemeen,1,GGD Regio Twente,0.552303,0.496977,0.607629,GGD Groningen,0.449671,False,True,0.102632
4,Gezondheid Algemeen,1,GGD Noord- en Oost-Gelderland,0.376585,0.338861,0.414309,GGD Groningen,0.449671,False,False,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
20,Totale Zorgkosten,6,GGD Limburg-Noord,2202.910436,1941.525155,2464.295717,GGD Zaanstreek/Waterland,2254.152562,False,False,0.000000
21,Totale Zorgkosten,6,GGD Zuid-Limburg,2142.851485,1888.592470,2397.110500,GGD Zaanstreek/Waterland,2254.152562,False,False,0.000000
22,Totale Zorgkosten,6,GGD Haaglanden,2217.889368,1954.726769,2481.051967,GGD Zaanstreek/Waterland,2254.152562,False,False,0.000000
23,Totale Zorgkosten,6,GGD Fryslân,2228.039226,1963.672301,2492.406152,GGD Zaanstreek/Waterland,2254.152562,False,False,0.000000
